In [3]:
from collections import defaultdict

import numpy as np
import pandas as pd
from sqlalchemy import create_engine


In [4]:
conn = create_engine(
    f"mysql+mysqlconnector://{'root'}:{'rootroot'}@{'127.0.0.1'}:{3306}/{'alaitp'}?charset=utf8", pool_recycle=3600)

In [5]:
def select_raw_words() -> pd.DataFrame:
    sql_query = """
                    SELECT * FROM keywords_en_core_web_lg
                """
    return pd.read_sql_query(sql_query, conn)

def select_job_model_words() -> pd.DataFrame:
    sql_query = """
                    SELECT * FROM keywords_job_model
                """
    return pd.read_sql_query(sql_query, conn)

In [6]:
df_raw = select_raw_words()
df_job = select_job_model_words()

In [7]:
df_job

,keyword_name,job_id,job_title,created_time,count,source,keyword_type
0,Data Engineer,00016610eea6e65ac752c90d65a45c51b0247cb8ed35cc...,Data Engineer,2020-01-17 20:23:06,1,dice,POSITION
1,Sql,00016610eea6e65ac752c90d65a45c51b0247cb8ed35cc...,Data Engineer,2020-01-17 20:23:06,1,dice,OTHER_LANGUAGE
2,Hive,00016610eea6e65ac752c90d65a45c51b0247cb8ed35cc...,Data Engineer,2020-01-17 20:23:06,2,dice,DATA_STORAGE
3,TeraData,00016610eea6e65ac752c90d65a45c51b0247cb8ed35cc...,Data Engineer,2020-01-17 20:23:06,1,dice,DATA_TRANSMISSION
4,Sunnyvale,00016610eea6e65ac752c90d65a45c51b0247cb8ed35cc...,Data Engineer,2020-01-17 20:23:06,1,dice,GPE
...,...,...,...,...,...,...,...
1587747,Canada,fffd98db79eed29774ed661bc777edfd4fc30d614944e8...,Illustrator,2020-01-18 08:36:40,1,dice,GPE
1587748,2019,fffd98db79eed29774ed661bc777edfd4fc30d614944e8...,Illustrator,2020-01-18 08:36:40,1,dice,CARDINAL
1587749,Robert Half Company,fffd98db79eed29774ed661bc777edfd4fc30d614944e8...,Illustrator,2020-01-18 08:36:40,1,dice,ORG
1587750,Equal Opportunity,fffd98db79eed29774ed661bc777edfd4fc30d614944e8...,Illustrator,2020-01-18 08:36:40,1,dice,OFFER


In [14]:
unique_tag = list(df_job.keyword_name.unique())
tag_idx = [unique_tag.index(word) for word in unique_tag]
tag_dict = dict(zip(unique_tag, tag_idx))
idx_to_tag = dict(zip(tag_idx, unique_tag))


In [15]:
entity_entity_matrix = np.zeros((len(unique_tag), len(unique_tag)), np.float64)
keyword_dict = defaultdict(list)
for row in df_job.itertuples():
    # Check whether the news_id exist
    keyword_tuple = (row.keyword_name, row.count, row.keyword_type)
    job_id = row.job_id
    keyword_dict[job_id].append(keyword_tuple)

for key in keyword_dict:
    for item in keyword_dict[key]:
        row_idx = tag_dict[item[0]]
        for word in keyword_dict[key]:
            col_idx = tag_dict[word[0]]
            entity_entity_matrix[row_idx, col_idx] += 1

In [16]:
def get_most_related_words(word: str, n: int) -> dict:
    """ word: the query word
        n: top n
    """
    top_n_dict = {}
    try:
        word_index = tag_dict[word]
    except KeyError:  # the word does not exist
        return top_n_dict
    # Sort the index by value, return indices of the highest value to the lowest
    top_n_indices = np.argsort(entity_entity_matrix[word_index])[::-1][0:n]
    top_n_counts = entity_entity_matrix[word_index][top_n_indices]
    for i, idx in enumerate(top_n_indices):
        tag = idx_to_tag[idx]
#         if tag != word:  # remove word itself
        top_n_dict[tag] = top_n_counts[i]
    return top_n_dict

In [25]:
get_most_related_words('Django', 50)

{'Django': 212.0,
 'Python': 191.0,
 'experience': 165.0,
 'Experience': 119.0,
 'AWS': 105.0,
 'design': 97.0,
 'Flask': 85.0,
 'SQL': 74.0,
 'architecture': 71.0,
 'JavaScript': 67.0,
 'web': 64.0,
 'Java': 57.0,
 'Agile': 56.0,
 'Linux': 56.0,
 'Equal Opportunity': 54.0,
 'Computer Science': 51.0,
 'API': 51.0,
 'React': 50.0,
 'Developer': 49.0,
 'developing': 49.0,
 'security': 43.0,
 'HTML': 42.0,
 'CI': 41.0,
 'the United States': 40.0,
 'Docker': 40.0,
 'CSS': 40.0,
 'DevOps': 39.0,
 'U.S.': 39.0,
 'CD': 39.0,
 'Git': 38.0,
 'best practices': 38.0,
 'agile': 37.0,
 'testing': 37.0,
 'PostgreSQL': 35.0,
 'web applications': 34.0,
 'MySQL': 33.0,
 'designing': 32.0,
 'REST': 32.0,
 'software development': 32.0,
 'APIs': 31.0,
 'Oracle': 31.0,
 'Angular': 31.0,
 'integration': 30.0,
 'backend': 30.0,
 'Javascript': 29.0,
 'Jenkins': 29.0,
 'Amazon Web': 28.0,
 'computer science': 28.0,
 'MongoDB': 28.0,
 'development': 27.0}